In [2]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 93.2 MB/s eta 0:00:00


In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import random

T = 5

normal_labels = ['O', 'B', 'I']
num_normal = len(normal_labels)

start_token = 'START'
stop_token = 'STOP'

label_to_index = {label: idx for idx, label in enumerate(normal_labels)}
B_idx = label_to_index['B']
O_idx = label_to_index['O']
I_idx = label_to_index['I']

model = LpProblem("sequence-labeling", LpMaximize)

x = {}
for j in range(num_normal):
    x[0, start_token, j] = LpVariable(f"x_0_START_{j}", cat="Binary")

for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            x[t, i, j] = LpVariable(f"x_{t}_{i}_{j}", cat="Binary")

for i in range(num_normal):
    x[T-1, i, stop_token] = LpVariable(f"x_{T-1}_{i}_STOP", cat="Binary")

s = {}
for j in range(num_normal):
    s[0, start_token, j] = random.uniform(-1, 1)

for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            s[t, i, j] = random.uniform(-1, 1)

for i in range(num_normal):
    s[T-1, i, stop_token] = random.uniform(-1, 1)


model += lpSum(s[key] * x[key] for key in x)


model += lpSum(x[0, start_token, j] for j in range(num_normal)) == 1

for t in range(1, T-1):
    model += lpSum(x[t, i, j] for i in range(num_normal) for j in range(num_normal)) == 1

model += lpSum(x[T-1, i, stop_token] for i in range(num_normal)) == 1


for j in range(num_normal):
    model += x[0, start_token, j] == lpSum(x[1, j, k] for k in range(num_normal))

for t in range(1, T-2):
    for label in range(num_normal):
        model += lpSum(x[t, i, label] for i in range(num_normal)) == lpSum(x[t+1, label, j] for j in range(num_normal))

for label in range(num_normal):
    model += lpSum(x[T-2, i, label] for i in range(num_normal)) == x[T-1, label, stop_token]


for t in range(1, T-1):
    model += x[t, O_idx, I_idx] == 0


for j in range(num_normal):
    if j != B_idx:
        model += x[0, start_token, j] == 0


model.solve()

print("🔹 Optimal Sequence Labeling Path:")

for j in range(num_normal):
    if x[0, start_token, j].value() == 1:
        print(f"Time step 0: {start_token} -> {normal_labels[j]}")

for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            if x[t, i, j].value() == 1:
                print(f"Time step {t}: {normal_labels[i]} -> {normal_labels[j]}")

for i in range(num_normal):
    if x[T-1, i, stop_token].value() == 1:
        print(f"Time step {T-1}: {normal_labels[i]} -> {stop_token}")


🔹 Optimal Sequence Labeling Path:
Time step 0: START -> B
Time step 1: B -> B
Time step 2: B -> I
Time step 3: I -> O
Time step 4: O -> STOP


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary
import random


n = 5
words = ["$", "w1", "w2", "w3", "w4", "w5"]


model = LpProblem("DependencyParsing", LpMaximize)


x = {}
for i in range(n+1):
    for j in range(1, n+1):
        x[(i, j)] = LpVariable(f"x_{i}_{j}", cat=LpBinary)


for j in range(1, n+1):
    model += x[(j, j)] == 0

for j in range(1, n+1):
    model += lpSum(x[(i, j)] for i in range(n+1)) == 1


score = {}
for i in range(n+1):
    for j in range(1, n+1):
        score[(i, j)] = random.uniform(-1, 1)

model += lpSum(score[(i, j)] * x[(i, j)] for i in range(n+1) for j in range(1, n+1))


for i in range(n+1):
    for j in range(i+1, n+1):
        if j - i - 1 > 0:
            model += (j - i - 1) * x[(i, j)] <= lpSum(x[(p, k)] for k in range(i+1, j) for p in range(i, j+1))


model.solve()


print("Dependency Parsing Tree:")
for j in range(1, n+1):
    for i in range(n+1):
        if x[(i, j)].varValue == 1:
            print(f"Parent of {words[j]} is {words[i]}")


Dependency Parsing Tree:
Parent of w1 is $
Parent of w2 is $
Parent of w3 is $
Parent of w4 is w5
Parent of w5 is w3
